In [1]:
pip install mysqlclient

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np

In [4]:
from env import host, user, password

url = f'mysql+pymysql://{user}:{password}@{host}/employees'

In [5]:
query = """SELECT * FROM salaries ORDER BY salary DESC LIMIT 10"""
df = pd.read_sql(query, url)
df

,emp_no,salary,from_date,to_date
0,43624,158220,2002-03-22,9999-01-01
1,43624,157821,2001-03-22,2002-03-22
2,254466,156286,2001-08-04,9999-01-01
3,47978,155709,2002-07-14,9999-01-01
4,253939,155513,2002-04-11,9999-01-01
5,109334,155377,2000-02-12,2001-02-11
6,109334,155190,2002-02-11,9999-01-01
7,109334,154888,2001-02-11,2002-02-11
8,109334,154885,1999-02-12,2000-02-12
9,80823,154459,2002-02-22,9999-01-01


In [6]:
# 1 Load the mpg dataset. Read the documentation for it, and use the data to answer these questions:
from pydataset import data
mpg = data('mpg') # load the dataset and store it in a variable
# data('mpg', show_doc=True) # view the documentation for the dataset
cars = pd.DataFrame(mpg)

In [8]:
# On average, which manufacturer has the best miles per gallon?
print(cars.info())
manufacturers = cars.groupby("manufacturer").mean()
cars['average_mileage'] = (cars.cty + cars.hwy) / 2
best_mpg = manufacturers.sort_values(by='average_mileage', ascending = False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 234 entries, 1 to 234
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   manufacturer     234 non-null    object 
 1   model            234 non-null    object 
 2   displ            234 non-null    float64
 3   year             234 non-null    int64  
 4   cyl              234 non-null    int64  
 5   trans            234 non-null    object 
 6   drv              234 non-null    object 
 7   cty              234 non-null    int64  
 8   hwy              234 non-null    int64  
 9   fl               234 non-null    object 
 10  class            234 non-null    object 
 11  average_mileage  234 non-null    float64
dtypes: float64(2), int64(4), object(6)
memory usage: 23.8+ KB
None


In [9]:
best_mpg.head(1)

,displ,year,cyl,cty,hwy,average_mileage
manufacturer,,,,,,
honda,1.711111,2003.0,4.0,24.444444,32.555556,28.5


In [10]:
#1 B.How many different manufacturers are there?
len(cars["manufacturer"].value_counts())


15

In [11]:
#1 C. How many different models are there?
len(cars["model"].value_counts())

38

In [12]:
#1 D. Do automatic or manual cars have better miles per gallon? Manual trans has better mpg
automatic = cars[cars.trans.str.contains("auto")]
print(automatic.average_mileage.mean())
manual = cars[cars.trans.str.contains("m")]
print(manual.average_mileage.mean())



19.130573248407643
22.227272727272727


In [ ]:
#2 Joining and Merging
# Copy the users and roles dataframes from the examples above. What do you think a right join 
# would look like? An outer join? What happens if you drop the foreign keys from the dataframes 
# and try to merge them?
users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles
# Right Join shows all the columns in the right data frame. If data is missing on the left DF
#NaN will populate
users_w_roles = pd.merge(users, roles, left_on='role_id', right_on='id', how='right')
users_w_roles

In [ ]:
#Outer Join
users_w_roles = pd.merge(users, roles, left_on='role_id', right_on='id', how='outer')
users_w_roles

In [ ]:
# What happens if you drop the foreign keys from the dataframes and try to merge them?
users = users.drop(columns=['role_id'])
users
pd.merge(users, roles, left_on = 'role_id', right_on='id') # cannot merge as we dropped role_id and 
                                                            # cannot merge on the role_id

In [13]:
#3 Getting data from SQL databases
# A.Create a function named get_db_url. It should accept a username, hostname, password, and 
# database name and return a url formatted like in the examples in this lesson.

def get_db_url(db):   ##Why does db need to be in the function? It will not work otherwise
    from env import host, user, password
    url = (f'mysql+pymysql://{user}:{password}@{host}/{db}')
    return url

In [14]:
# Showing all databases
query = '''show databases'''
pd.read_sql(query, url)

,Database
0,information_schema
1,albums_db
2,chipotle
3,darden_1043
4,elo_db
5,employees
6,fruits_db
7,iris_db
8,join_example_db
9,mall_customers


In [ ]:
# 3B. Use your function to obtain a connection to the employees database.
pd.read_sql('SELECT * FROM employees LIMIT 5 OFFSET 50', url)
# or
pd.read_sql('SELECT * FROM employees LIMIT 5 OFFSET 50', get_db_url('employees'))

In [ ]:
#3C. Intentionally make an error in your SQL query. What does the error message look like?
def bad_url(un,pas,hos,db):
    url = (f'mysql+pymysql://{user}:{password}@{host}/{db}')
    return url
pd.read_sql('SELECT * FROsM employees LIMIT 5 OFFSET 50', bad_url(user, host, password, 'employees'))

# w/ bad FROsM statement 
#ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'FROsM employees LIMIT 5 OFFSET 50' at line 1")
#[SQL: SELECT * FROsM employees LIMIT 5 OFFSET 50]
#(Background on this error at: http://sqlalche.me/e/f405)


In [ ]:
#3D. Read the employees and titles tables into two separate dataframes
employees = pd.read_sql('SELECT * FROM employees', get_db_url('employees'))
employees
titles = pd.read_sql('SELECT * FROM titles', get_db_url('employees'))
titles

In [ ]:
#3E. Visualize the number of employees with each title.
sql = """SHOW TABLES"""
pd.read_sql(sql, url)
print(titles.head(5)) # titles has emp_no, title, from_date, & to_date
print(employees.head(5)) #employees has emp_no, birth_date, first_name, last_name, gender and hire_date

In [ ]:
#going to join tables to get a count of employee titles
current_titles = pd.read_sql('SELECT * FROM titles WHERE to_date > CURDATE()', url)
current_titles # getting current titles WHERE to_date > CURDATE()
#Joining current titles with employees
emp_w_current_titles = pd.merge(current_titles, employees, left_on='emp_no', right_on='emp_no', how='right')
#value counts shows us the counts of the distinct title names
#getting value count of title on the merged tables
emp_w_current_titles.title.value_counts()

In [ ]:
#Alt way to get current employees titles
cur_titles = titles[titles['to_date'] >= pd.to_datetime('today')] # current employees titles
cur_titles
#grouping by title in current titles. Doing a count on the emp_no
count_titles = cur_titles.groupby('title').emp_no.agg(['count']) 
count_titles

In [ ]:
#3F. Join the employees and titles dataframes together.
# merging employees and titles on emp_no, showing all columns of data in each
print(titles.head)
print(employees.head)
emp_w_titles = pd.merge(employees, titles, left_on = 'emp_no', right_on = 'emp_no', how = "inner")
emp_w_titles

In [ ]:
#3G Visualize how frequently employees change titles.

In [ ]:
#3H. For each title, find the hire date of the employee that was hired most recently with that title..max
emp_w_titles.groupby('title')['hire_date'].max()

In [ ]:
#3H alt
titles.groupby('title').from_date.max()

In [ ]:
#3I. Write the code necessary to create a cross tabulation of the number of titles by department. 
# (Hint: this will involve a combination of SQL and python/pandas code)
#Making dept emp DF to get dept names to tie to titles
dept_emp = pd.read_sql('SELECT * FROM dept_emp', get_db_url('employees'))
dept_emp
# Making depts DF to get departments from dept number
depts = pd.read_sql('SELECT * FROM dept_emp', get_db_url('employees'))
depts
# Current titles DF. Created up top
cur_titles = titles[titles['to_date'] >= pd.to_datetime('today')]
# Current departments DF
cur_dept = dept_emp[dept_emp['to_date'] >= pd.to_datetime('today')]
# Joining depts and cur_dept to see current departments along with department numbers
depts_n_cur_depts = pd.merge(depts,cur_dept, left_on = 'dept_no', right_on = "dept_no", how = 'inner')
# Joining depts_n_cur_depts with cur_titles
depts_w_titles = pd.merge(depts_n_cur_depts, cur_titles, left_on = 'emp_no', right_on = 'emp_no', how = 'inner')
# Crosstab of dept_name and titles
depts_w_titles_crossstab = pd.crosstab(dept_w_titles.dept_name,depts_w_titles.titles)
depts_w_titles_crosstab



In [19]:
# 4 Use your get_db_url function to help you explore the data from the chipotle database. 
# Use the data to answer the following questions:
orders = pd.read_sql('''SELECT * FROM orders''',get_db_url('chipotle'))
orders.head()

#A What is the total price for each order?
#Making item price to show as a float
orders['item_price'] = orders.item_price.str.replace('$','').astype(float)
# sum up the total of quantity  * item price
orders["total"] = orders.quantity * orders.item_price
orders

,id,order_id,quantity,item_name,choice_description,item_price,total
0,1,1,1,Chips and Fresh Tomato Salsa,nan,2.39,2.39
1,2,1,1,Izze,[Clementine],3.39,3.39
2,3,1,1,Nantucket Nectar,[Apple],3.39,3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,2.39,2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",16.98,33.96
...,...,...,...,...,...,...,...
4617,4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour Cream, Cheese, Lettuce, Guacamole]]",11.75,11.75
4618,4619,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese, Lettuce, Guacamole]]",11.75,11.75
4619,4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto Beans, Guacamole, Lettuce]]",11.25,11.25
4620,4621,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettuce]]",8.75,8.75


In [20]:
#B What are the most popular 3 items?
orders.item_name.value_counts().head(3)



Chicken Bowl           726
Chicken Burrito        553
Chips and Guacamole    479
Name: item_name, dtype: int64

In [30]:
#C Which item has produced the most revenue?
totals = orders.groupby('item_name').total.sum()
totals
round(totals.max(),2)

item_name
6 Pack Soft Drink                         369.93
Barbacoa Bowl                             672.36
Barbacoa Burrito                          894.75
Barbacoa Crispy Tacos                     138.71
Barbacoa Salad Bowl                       106.40
Barbacoa Soft Tacos                       250.46
Bottled Water                             649.18
Bowl                                       74.00
Burrito                                    44.40
Canned Soda                               191.84
Canned Soft Drink                         603.75
Carnitas Bowl                             830.71
Carnitas Burrito                          616.33
Carnitas Crispy Tacos                      95.94
Carnitas Salad                              8.99
Carnitas Salad Bowl                        66.34
Carnitas Soft Tacos                       375.94
Chicken Bowl                             8044.63
Chicken Burrito                          6387.06
Chicken Crispy Tacos                      524.11
Chicken Sa

In [25]:
type(totals)

pandas.core.series.Series

KeyError: 8044.629999999975